In [1]:
import sys
sys.path.append('/mnt/antares_raid/home/oliver/nideep')
sys.path.append('/mnt/antares_raid/home/oliver/Scripts/autoencoder')
import os
from TransformMNIST import MNISTtransformer
import subprocess
import sys
import caffe
import matplotlib
import numpy as np
import lmdb
import argparse
from collections import defaultdict
import matplotlib.pyplot as plt
import math
from nideep.eval.learning_curve import LearningCurve
from nideep.eval.inference import infer_to_h5_fixed_dims, infer_to_lmdb
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
%matplotlib inline

/mnt/antares_raid/home/oliver/adhara/src/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Net<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/mnt/antares_raid/home/oliver/adhara/src/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Blob<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \
/mnt/antares_raid/home/oliver/adhara/src/caffe/python/caffe/pycaffe.py:13: RuntimeWarning: to-Python converter for boost::shared_ptr<caffe::Solver<float> > already registered; second conversion method ignored.
  from ._caffe import Net, SGDSolver, NesterovSolver, AdaGradSolver, \


In [ ]:
def set_up_dir(path):
    try: 
        os.makedirs(path)
    except OSError:
        if not os.path.isdir(path):
            raise

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [ ]:
src_lmdb_dir = '/mnt/antares_raid/home/oliver/adhara/src/caffe/examples/mnist/'
home_dir = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/'
set_up_dir(home_dir)
dst_lmdb_dir = home_dir + 'MNIST_lmdb/'
set_up_dir(dst_lmdb_dir)
N_ims = [60000, 10000]
angles = range(-90,90+1,15)
batch_size = 1000.
set_strings = ['_TRAIN_', '_TEST_']
dir_strings = ['_train_', '_test_']

if False:
    # Create the lmdb database
    for set_str, dir_str, N_im in zip(set_strings, dir_strings,N_ims):
        mnisttransform = MNISTtransformer(src_lmdb_dir, home_dir, dst_lmdb_dir, N_im, batch_size, set_str, dir_str, angles)
        mnisttransform.run()

In [ ]:
# Train autoencoder weights
#p = subprocess.Popen(["screen"])
#p = subprocess.Popen(["/mnt/antares_raid/home/oliver/adhara/src/caffe/build/tools/caffe", "train", "-solver", "/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder/autoencoder_net_solver.prototxt"])

#/mnt/antares_raid/home/oliver/adhara/src/caffe/build/tools/caffe train -solver /mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder/autoencoder_net_solver.prototxt 

In [ ]:

# Check weights

proto = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder/deploy_autoencoder_net.prototxt'
model = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder/snapshots/_iter_156000.caffemodel'
lmdb_path = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/MNIST_lmdb/MNIST_TEST_10000_rot_lmdb/shuffled/'

proto = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder_with_MLP/deploy_autoencoder_with_MLP_net.prototxt'
model = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder_with_MLP/snapshots_with_MLP/_iter_780000.caffemodel'
lmdb_path = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/MNIST_lmdb/MNIST_TEST_10000_rot_lmdb/shuffled/'

net = caffe.Net(proto, model, caffe.TEST)


count = 0
correct = 0
matrix = defaultdict(int) # (real,pred) -> int
labels_set = set()

net = caffe.Net(proto, model, caffe.TEST)
caffe.set_mode_cpu()
lmdb_env = lmdb.open(lmdb_path)
lmdb_txn = lmdb_env.begin()
lmdb_cursor = lmdb_txn.cursor()
for key, value in lmdb_cursor:
    datum = caffe.proto.caffe_pb2.Datum()
    datum.ParseFromString(value)
    label = int(datum.label)
    image = caffe.io.datum_to_array(datum)
    image = image.astype(np.uint8)
    
    out = net.forward_all(data=np.asarray([image]))
    plabel = int(out['score'][0].argmax(axis=0))

    count = count + 1
    iscorrect = label == plabel
    correct = correct + (1 if iscorrect else 0)
    matrix[(label, plabel)] += 1
    labels_set.update([label, plabel])

    #if not iscorrect:
    #    print("\rError: key=%s, expected %i but predicted %i" \
    #            % (key, label, plabel))

    #sys.stdout.write("\rAccuracy: %.1f%%" % (100.*correct/count))
    #sys.stdout.flush()
    
    plt.figure(figsize=(18,4))
    plt.subplot(1,2,1)
    plt.imshow(image.reshape(28, 28).T, cmap='gray'); plt.axis('off')
    plt.colorbar()
    plt.title('Input')
    plt.subplot(1,2,2)
    plt.imshow(np.array([sigmoid(a) for a  in out['decode1'].squeeze()]).reshape(28, 28).T, cmap='gray'); plt.axis('off')
    plt.title('decode1 output')
    plt.colorbar()
    plt.show()

    if count == 5:
        break
#print(str(correct) + " out of " + str(count) + " were classified correctly")

print ""
print "Confusion matrix:"
print "(r , p) | count"
for l in labels_set:
    for pl in labels_set:
        print "(%i , %i) | %i" % (l, pl, matrix[(l,pl)])

In [ ]:
#Train Autoencoder + MLP
model = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder/snapshots_28_07/_iter_702000.caffemodel'

#p = subprocess.Popen(["screen"])
#p = subprocess.Popen(["/mnt/antares_raid/home/oliver/adhara/src/caffe/build/tools/caffe", "train", 
#"-solver", "/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder_with_MLP/autoencoder_with_MLP_solver.prototxt","-weights", "/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder/snapshots_28_07/_iter_702000.caffemodel"])


In [2]:
# Use Youssef Script to save results to hdf5
proto = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder_with_MLP/autoencoder_with_MLP_net.prototxt'
model = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder_with_MLP/snapshots_with_MLP/_iter_780000.caffemodel'
lmdb_path = '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/MNIST_lmdb/MNIST_TRAIN_60000_rot_lmdb/shuffled/'

net = caffe.Net(proto, model, caffe.TRAIN)
#print(list(net._layer_names))

keys = ['label', 'score', 'data']
dst_fpath =  "/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder_with_MLP/results/res_train.hdf5"
n = 780
infer_to_h5_fixed_dims(net, keys, n, dst_fpath, preserve_batch=False)

# Use encoding weights to train an MLP

dst_prefix =  '/mnt/antares_raid/home/oliver/Scripts/autoencoder_v2/autoencoder_with_MLP/results/%s'
print("blobs {}\nparams {}".format(net.blobs.keys(), net.params.keys()))



blobs ['data', 'label', 'data_unrot', 'label_unrot', 'flatdata_unrot', 'encode1', 'encode1neuron', 'encode2', 'encode2neuron', 'encode3', 'encode3neuron', 'encode4', 'encode4neuron', 'score']
params ['encode1', 'encode2', 'encode3', 'encode4', 'score']


In [4]:
import h5py

f = h5py.File(dst_fpath, "r")
#for name in f:
#    a = f[name]

In [5]:
pred = [np.argmax(p) for p in f['score']]
true = list(f['label']) 
print(classification_report(true,pred))
confusion_matrix(true,pred)

             precision    recall  f1-score   support

        0.0       0.98      0.98      0.98     76999
        1.0       0.98      0.98      0.98     87646
        2.0       0.94      0.95      0.94     77454
        3.0       0.94      0.92      0.93     79703
        4.0       0.94      0.95      0.94     75946
        5.0       0.93      0.93      0.93     70473
        6.0       0.95      0.95      0.95     76934
        7.0       0.94      0.93      0.94     81445
        8.0       0.93      0.94      0.93     76063
        9.0       0.91      0.90      0.91     77337

avg / total       0.94      0.94      0.94    780000



array([[75518,    12,   196,   132,    14,   226,   360,    32,   378,
          131],
       [    0, 86030,   540,   196,   220,    65,    53,   213,   280,
           49],
       [  291,   233, 73308,   459,   717,    69,   201,  1177,   880,
          119],
       [  189,   133,  1226, 73413,    76,  1505,   278,   602,  1676,
          605],
       [   74,   123,   370,   176, 71833,   548,   336,   429,   499,
         1558],
       [  337,   106,   164,   999,   452, 65518,  1147,   484,   650,
          616],
       [  281,   141,   235,   239,   262,   862, 73011,   240,   304,
         1359],
       [  142,   435,  1264,   297,  1021,   313,   154, 76072,   174,
         1573],
       [  204,   540,   615,  1222,   706,   502,   357,   225, 71139,
          553],
       [  408,   222,   243,   944,  1345,   891,  1263,  1377,   800,
        69844]])

In [ ]:
pred